## Importing Computer Vision Libraries

In [43]:
import torch
from torch import nn

import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

from torch.utils.data import DataLoader

import time


In [4]:
device = "cuda" if torch.cuda.is_available() else 'cpu'

## Preparing the data

In [6]:
#training data

train_data = datasets.FashionMNIST(root='data', train= True, transform= ToTensor(), download= True)
test_data = datasets.FashionMNIST(root='data', train= False, transform= ToTensor(), download= True)

In [ ]:
print(f" the length of training data is {len(train_data)}, length of test data  { len(test_data)}")

In [ ]:
#visualize any image randomly
img, label = train_data[2]
img, label

In [ ]:
classnames = train_data.classes
classnames

In [ ]:
#visualize the image

plt.imshow(img.squeeze(), cmap="gray")
plt.title(classnames[label])
plt.axis(False)



In [ ]:
#multiple random images
torch.manual_seed(42)
fig  = plt.Figure(figsize=(9,9))
rows, cols = 4, 4

for i in range(1, rows*cols +1):
    #pick image randomly
    random_idx = torch.randint(0, len(train_data), size=[1]).item()
    img, label = train_data[random_idx]
    fig.add_subplot(rows, cols, i)
    plt.imshow(img.squeeze(), cmap = 'gray')
    plt.axis(False)
    plt.title(classnames[label])

In [ ]:
#dataloader to loop through the dataset

#hyperparameter
BATCH_SIZE = 32

train_data_batch = DataLoader(train_data, batch_size = BATCH_SIZE, shuffle= True )

test_data_batch = DataLoader(test_data, batch_size = BATCH_SIZE, shuffle= False)

print(f" The reduced training databatch is {len(train_data_batch)}")
print(f" The reduced test databatch is {len(test_data_batch)}")

In [33]:
# Check out what's inside the training dataloader
train_features_batch, train_labels_batch = next(iter(train_data_batch))
train_features_batch.shape, train_labels_batch.shape

(torch.Size([32, 1, 28, 28]), torch.Size([32]))

## Creating a baseline model

In [37]:
class FashionMNISTVO(nn.Module):
    def __init__(self, input_shape: int, hidden_layers: int,  output_shape: int):
        super().__init__()

        self.linear_stack = nn.Sequential( nn.Flatten(),
                                          nn.Linear(in_features= input_shape, out_features= hidden_layers),
                                          nn.Linear(in_features= hidden_layers, out_features= output_shape),
                                          )
        

    #define the forward method
    def forward(self, x):
        return self.linear_stack(x)

In [39]:
#create a model for the class

model_0 = FashionMNISTVO(input_shape= 784,
                         hidden_layers= 10,
                         output_shape=len(classnames)).to('cpu')

model_0

FashionMNISTVO(
  (linear_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
  )
)

## Setup loss, optimizer and evaluation metrics

In [40]:
#loss function

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_0.parameters(), lr = 0.1)

In [42]:
#import helper functions

import requests
from pathlib import Path

r = requests.get(url='https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py')

if Path("helper_functions.py").is_file():
    print(" Skipping the download the file already exists")
else:
    print('Downloading file...........')

    with open('helper_functions.py', 'wb') as f:

        f.write(r.content)

 Skipping the download the file already exists


In [44]:
#time function

def my_run_time(start_time, end_time):

    return f" the elapsed time is {start_time - end_time} seconds"
    